In [ ]:
!pip install transformers ipython pdfkit

In [ ]:
!apt-get install wkhtmltopdf

In [33]:
from transformers import pipeline, set_seed
from IPython.display import display, HTML
import torch
import pdfkit
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
precision = 'full' # @param ["half", "full"]
if precision == "half":
    selected_precision = torch.float16
elif precision == "full":
    selected_precision = torch.float32

In [5]:
model_name = "/content/drive/MyDrive/nlp-model/distilgpt2-indo-webtext-mp-fp16"

In [6]:
seed = 3693 # @param {type:"integer"}
set_seed(seed)

In [7]:
generator = pipeline('text-generation',
                     model=model_name,
                     torch_dtype=selected_precision)

In [30]:
prompt = "De-escalasi konflik adalah" # @param {type:"string"}

In [31]:
preds = generator(prompt,
                  max_length=64,
                  num_return_sequences=4)

In [32]:
html_code = f"""
<!DOCTYPE html>
<html>
<head>
</head>
<body>
<p>Prompt: {prompt}</p>
<p>seed: {seed}</p>
<div style='display: inline-block; border: 2px solid black; padding: 10px; width: 300px; height: 300px; overflow: auto;'>
    <p style='font-family: monospace;'>{preds[0]['generated_text']}</p>
</div>
<div style='display: inline-block; border: 2px solid black; padding: 10px; width: 300px; height: 300px; overflow: auto;'>
    <p style='font-family: monospace;'>{preds[1]['generated_text']}</p>
</div>
<div style='display: inline-block; border: 2px solid black; padding: 10px; width: 300px; height: 300px; overflow: auto;'>
    <p style='font-family: monospace;'>{preds[2]['generated_text']}</p>
</div>
<div style='display: inline-block; border: 2px solid black; padding: 10px; width: 300px; height: 300px; overflow: auto;'>
    <p style='font-family: monospace;'>{preds[3]['generated_text']}</p>
</div>
</body>
</html>
"""

# Display the styled output
display(HTML(html_code))

In [34]:
base_name = 'output'
html_file = f'{base_name}.html'
pdf_file = f'{base_name}.pdf'
count = 1

In [41]:
count = 1  # Start with count 1
while True:
    html_file = f"{base_name}_{count}.html"
    if not os.path.exists(html_file):  # Check if file doesn't exist
        break  # Exit loop if file doesn't exist (available to write)
    count += 1

with open(html_file, 'w') as f:
    f.write(html_code)
print(f"HTML saved as {html_file}")

HTML saved as output_2.html


In [43]:
count = 1
while True:
    html_file = f"{base_name}_{count}.html"  # Use .html extension for HTML file
    pdf_file = f"{base_name}_{count}.pdf"   # Separate variable for PDF filename
    if not os.path.exists(pdf_file):  # Check if PDF doesn't exist
        break
    count += 1

pdfkit.from_file(html_file, pdf_file)  # Generate PDF after finding unique filename

print(f"PDF saved as {pdf_file}")

PDF saved as output_2.pdf


In [44]:
import os
from zipfile import ZipFile

def zip_files(directory, zip_name):
  """
  Zips all HTML and PDF files in a directory.

  Args:
      directory: The directory path containing the files to zip.
      zip_name: The name of the output zip file.
  """
  with ZipFile(zip_name, 'w') as zip_file:
    for root, _, files in os.walk(directory):
      for file in files:
        if file.endswith(".html") or file.endswith(".pdf"):
          file_path = os.path.join(root, file)
          zip_file.write(file_path, os.path.relpath(file_path, directory))

# Replace 'path/to/your/directory' with the actual directory path
directory = os.getcwd()
zip_name = 'scientific_research_data.zip'

zip_files(directory, zip_name)

print(f"Zipped files from '{directory}' to '{zip_name}'.")

Zipped files from '/content' to 'scientific_research_data.zip'.
